In [7]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
print(os.getcwd())

c:\Users\utcpret\Documents\NF26\projet bges


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, date

import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql import Row

In [4]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
## Définition du fuseau horaire sur paris ##

spark.conf.set("spark.sql.session.timeZone", "Europe/Paris")


In [3]:
def lire_fichiers_txt_csv(dossier, delimiter=';', infer_schema=True, header=True):
    """
    Lit tous les fichiers .txt d'un dossier comme des CSV structurés (avec séparateur).
    
    Paramètres :
    - dossier : chemin vers le dossier contenant les fichiers .txt
    - delimiter : caractère séparateur (par défaut ';')
    - infer_schema : déduire automatiquement les types de colonnes (True/False)
    - header : les fichiers contiennent une ligne d'en-tête (True/False)
    
    Retour :
    - Spark DataFrame contenant toutes les lignes de tous les fichiers.
    """
    spark = SparkSession.builder.getOrCreate()
    dossier = dossier.replace('\\', '/')  # Compatibilité avec Spark
    
    sdf = spark.read.option("header", header) \
                    .option("inferSchema", infer_schema) \
                    .option("delimiter", delimiter) \
                    .csv(f"{dossier}/*.txt")
    return sdf

In [6]:
dossier = 'C:\Users\utcpret\Documents\NF26\projet bges\BDD_BGES\BDD_BGES_BERLIN\BDD_BGES_BERLIN_INFORMATIQUE'
sdf_berlin_info = lire_fichiers_txt_csv(dossier)


SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2504860167.py, line 1)